In [1]:
import h5py
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import matplotlib.cm as cm
from scipy.signal import savgol_filter
from scipy import stats

batch1 = pickle.load(open(r'D:\논문 준비\코드 복사\코드 복사\코드 복사\batch1.pkl', 'rb'))
#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

numBat1 = len(batch1.keys())

batch2 = pickle.load(open(r'D:\논문 준비\코드 복사\코드 복사\코드 복사\batch2.pkl','rb'))
# There are four cells from batch1 that carried into batch2, we'll remove the data from batch2
# and put it with the correct cell from batch1
batch2_keys = ['b2c7', 'b2c8', 'b2c9', 'b2c15', 'b2c16']
batch1_keys = ['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4']
add_len = [662, 981, 1060, 208, 482];

for i, bk in enumerate(batch1_keys):
    batch1[bk]['cycle_life'] = batch1[bk]['cycle_life'] + add_len[i]
    for j in batch1[bk]['summary'].keys():
        if j == 'cycle':
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j] + len(batch1[bk]['summary'][j])))
        else:
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j]))
    last_cycle = len(batch1[bk]['cycles'].keys())
    for j, jk in enumerate(batch2[batch2_keys[i]]['cycles'].keys()):
        batch1[bk]['cycles'][str(last_cycle + j)] = batch2[batch2_keys[i]]['cycles'][jk]

del batch2['b2c7']
del batch2['b2c8']
del batch2['b2c9']
del batch2['b2c15']
del batch2['b2c16']

numBat2 = len(batch2.keys())

batch3 = pickle.load(open(r'D:\논문 준비\코드 복사\코드 복사\코드 복사\batch3.pkl','rb'))
# remove noisy channels from batch3
del batch3['b3c37']
del batch3['b3c2']
del batch3['b3c23']
del batch3['b3c32']
del batch3['b3c38']
del batch3['b3c39']

numBat3 = len(batch3.keys())
numBat = numBat1 + numBat2 + numBat3
bat_dict = {**batch1, **batch2, **batch3}

In [2]:
bat_dict.keys()

dict_keys(['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4', 'b1c5', 'b1c6', 'b1c7', 'b1c9', 'b1c11', 'b1c14', 'b1c15', 'b1c16', 'b1c17', 'b1c18', 'b1c19', 'b1c20', 'b1c21', 'b1c23', 'b1c24', 'b1c25', 'b1c26', 'b1c27', 'b1c28', 'b1c29', 'b1c30', 'b1c31', 'b1c32', 'b1c33', 'b1c34', 'b1c35', 'b1c36', 'b1c37', 'b1c38', 'b1c39', 'b1c40', 'b1c41', 'b1c42', 'b1c43', 'b1c44', 'b1c45', 'b2c0', 'b2c1', 'b2c2', 'b2c3', 'b2c4', 'b2c5', 'b2c6', 'b2c10', 'b2c11', 'b2c12', 'b2c13', 'b2c14', 'b2c17', 'b2c18', 'b2c19', 'b2c20', 'b2c21', 'b2c22', 'b2c23', 'b2c24', 'b2c25', 'b2c26', 'b2c27', 'b2c28', 'b2c29', 'b2c30', 'b2c31', 'b2c32', 'b2c33', 'b2c34', 'b2c35', 'b2c36', 'b2c37', 'b2c38', 'b2c39', 'b2c40', 'b2c41', 'b2c42', 'b2c43', 'b2c44', 'b2c45', 'b2c46', 'b2c47', 'b3c0', 'b3c1', 'b3c3', 'b3c4', 'b3c5', 'b3c6', 'b3c7', 'b3c8', 'b3c9', 'b3c10', 'b3c11', 'b3c12', 'b3c13', 'b3c14', 'b3c15', 'b3c16', 'b3c17', 'b3c18', 'b3c19', 'b3c20', 'b3c21', 'b3c22', 'b3c24', 'b3c25', 'b3c26', 'b3c27', 'b3c28', 'b3c29', 'b3c30

In [3]:
bat_dict['b1c0'].keys()

dict_keys(['cycle_life', 'charge_policy', 'summary', 'cycles'])

In [6]:
bat_dict['b1c0']['charge_policy']

'3.6C(80%)-3.6C'

# policy 를 통해서 비대칭성을 확인할 수 있다. (Discharge 4C)

In [7]:
import re

In [8]:
policys0=bat_dict['b1c0']['charge_policy']
policy0=re.sub(r'[^0-9]', '', policys0)
print(policy0)

368036


# 여기서 4C(80%)-4C, 3.6(80%)-3.6C, 4.8C(80%)-4.8C 를 비교( 같은 날짜로)

In [29]:
policy3_6=[]
policy4=[]
policy4_8=[]
count=0
for i in bat_dict.keys():
    policys0=bat_dict[i]['charge_policy']
    if policys0[3]=='C' and policys0[-3]=='.': # 0.0C(%)-0.0C
        if policys0[:3]==policys0[-4:-1]=='3.6': #같은 C-rate 3.6
            print(i)
            policy3_6.append(i)
for i in bat_dict.keys():
    policys0=bat_dict[i]['charge_policy']
    if policys0[1]=='C' and policys0[-3]=='-': # 0C(%)-0C
        if policys0[0]==policys0[-2]=='4': #같은 C-rate 4
            print(i)
            policy4.append(i)
for i in bat_dict.keys():
    policys0=bat_dict[i]['charge_policy']
    if policys0[3]=='C' and policys0[-3]=='.': # 0.0C(%)-0.0C
        if policys0[:3]==policys0[-4:-1]=='4.8': #같은 C-rate 4.8
            print(i)
            policy4_8.append(i)

b1c0
b1c1
b1c2
b1c3
b1c4
b1c6
b1c7
b2c24
b2c25
b2c26


In [31]:
policy3_6 , policy4, policy4_8# 4.8은 b1c6, b1c7 사용

(['b1c0', 'b1c1', 'b1c2'],
 ['b1c3', 'b1c4'],
 ['b1c6', 'b1c7', 'b2c24', 'b2c25', 'b2c26'])

In [38]:
#3.6
for i in policy3_6:
    a= bat_dict[i]['cycle_life'][0]
    print(f'3.6C : {a}')
#4
for i in policy4:
    a= bat_dict[i]['cycle_life'][0]
    print(f'4C : {a}')
#4.8
for i in policy4_8[:2]:
    a= bat_dict[i]['cycle_life'][0]
    print(f'4.8C : {a}')

3.6C : [1852.]
3.6C : [2160.]
3.6C : [2237.]
4C : [1434.]
4C : [1709.]
4.8C : [636.]
4.8C : [870.]


In [ ]:
# 제공된 데이터는 3.6C~ 4.8C Charge C-rate로 급속충전에 해당. 방전은 4C-rate 급속방전에 해당되어
# 비교할 수 없음. 이 데이터를 바탕으로는 급속 충.방전 시 높은 C-rate 값을 갖는다면 짧은 Cycle life 를 가짐.

In [40]:
print(numBat2),print(numBat1)

43
41


(None, None)